In [1]:
import sys
sys.version

'3.12.0 (tags/v3.12.0:0fb18b0, Oct  2 2023, 13:03:39) [MSC v.1935 64 bit (AMD64)]'

In [1]:
pip install --upgrade pip

In [1]:
pip install numpy scipy

In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.
